# Abigail Hotaling
## Multi-armed bandits algorithm 
$\epsilon$ WGX: Adaptive Edge Probing for Enhancing Incomplete Networks 

In [162]:
import networkx as nx
import random
import matplotlib.pyplot as plt
from sympy.solvers import solve
from sympy import Symbol
import sympy as sy

In [191]:
def probe(select, sample, data, node_id, reward_total, reward_obs, reward, C, probed):
    # probe select 
    possible_nodes = list(data.neighbors(select))
    result = random.choice(possible_nodes)
                    
        # duplicate node? 
    if sample.has_node(result):
        distinct = False
        #update duplicate neighbor
        print(C[select])
        update = (C[select][0], C[select][1] + 1)
        C[select] = update
        print(C[select])
        #edge? 
        if sample.has_edge(select,result)==False:
            sample.add_edge(select,result)
            
        reward_obs[select] = (reward_total[select])/(C[select][0] + C[select][1])


            # else update distinct neighbor
    else:
        distinct=True
        update = (C[select][0] + 1, C[select][1])
        C[select] = update
        C[result] = (0, 0)
        sample.add_node(result)
        sample.add_edge(select,result)
        node_id.append(result)
                
        # update reward functions
        #observed reward 

        node_id.append(result)
        reward_obs[result] = 0
        reward[result] = 0
        reward_total[result] = 0
        reward_total[select] = reward_total[select]+1
        reward_obs[select] = (reward_total[select])/(C[select][0] + C[select][1])
                    
    # initialize p
    # if node has not been probed, or if node u has no duplicate neighbors, then p=1, otherwise p is solved from section 3.2
    # p = (MLE - obs.degree)/MLE
                
        #update p and expected reward
        k = C[select][0] + C[select][1]
        s = C[select][0]/k
        x = Symbol('x')
        solve_m = solve((1-sy.exp(-x))/x - s, x)
        if not solve_m:
            print('Solve for m failed, setting m = 1')
            print('s  = ' + str(s))
            m = 1
        else: 
            print('Correctly solved for m')
            print('s  = ' + str(s))
            m = solve_m[0]
    
        estimated_degree = k/m
        observed_degree = sample.degree(select)
                
        if select not in probed:
            p = 1 
                
        elif C[select][1]==0:
            p=1 
                    
        else:
            p = (estimated_degree-observed_degree)/estimated_degree

                
        reward[select] = reward_obs[select]*p
                
    return sample, node_id, reward_total, reward_obs, reward, distinct, C

def explore(sample, data, node_id, reward_total, reward_obs, reward, distinct, C, probed):
    # explore
    #select random node
    select = random.choice(node_id)
    # probe select 
    sample, node_id, reward_total, reward_obs, reward, distinct, C = probe(select, sample, data, node_id, reward_total, reward_obs, reward, C, probed)

    return select, sample, node_id, reward_total, reward_obs, reward, distinct, C

def exploit(sample, data, node_id, reward_total, reward_obs, reward, distinct, C, probed):
    #select best node --> based on expected reward
    max_reward = [i for i, e in enumerate(reward) if e == max(reward.values())]
    select = random.choice(max_reward)
    #probe select
    sample, node_id, reward_total, reward_obs, reward, distinct, C = probe(select, sample, data, node_id, reward_total, reward_obs, reward, C, probed)
    
    return select, sample, node_id, reward_total, reward_obs, reward, distinct, C

def densification(e1, sample, data, node_id, reward_total, reward_obs, reward, distinct, C, probed):
    if (random.randrange(0, 1)<e1): # explore # random.random() 
        select, sample, node_id, reward_total, reward_obs, reward, distinct, C = explore(sample, data, node_id, 
                                                                              reward_total, reward_obs,
                                                                              reward, distinct, C, probed)
        
    else: #exploit
        select, sample, node_id, reward_total, reward_obs, reward, distinct, C = exploit(sample, data, node_id, 
                                                                              reward_total, reward_obs,
                                                                              reward, distinct, C, probed)
    return select, sample, node_id, reward_total, reward_obs, reward, distinct, C
    
    
def expansion(sample, data, node_id, reward_total, reward_obs, reward, distinct, C, probed):
    expansion_choices = [i for i, e in enumerate(node_id) if ((e not in probed) and (e not in sample_original.nodes()))]

    select = random.choice(expansion_choices)
    #probe select
    sample, node_id, reward_total, reward_obs, reward, distinct, C = probe(select, sample, data, node_id, reward_total, reward_obs, reward, C, probed)
    return select, sample, node_id, reward_total, reward_obs, reward, distinct, C

In [192]:
def eWGX(e0, e1, sample, data, budget): 
    """
    Parameters
    ----------
    
    e0 : float
        probability for outer level bandit
    
    e1 : float 
        probability for innter level bandit
        
    sample : nx.graph
        graph of already observed data
        
    data : nx.graph
        graph of all data
        
    budget : int
        number of probes in budget
        
    Returns
    -------
    nx.graph
        Probed sample graph
        
    """
    
    #initialize rewards
    # reward_obs = average observed reward for each node
    reward_obs = dict.fromkeys(list(sample.nodes), 0)
    reward_total = dict.fromkeys(list(sample.nodes), 0)
    # C = (A_u, B_u) A_u distinct neighbors observed, B_u duplicate neighbors observed
    C = dict.fromkeys(list(sample.nodes()), (0, 0))
    # reward_exp = average reward observed when expansion
    reward_exp = 0
    num_exp = 0 # number of expansion probes 
    # reward_den = average reward observed when densification
    reward_den = 0
    num_den = 0 # number of densification probes 

    # reward = expected rewards, reward.obs*p
    reward = dict.fromkeys(list(sample.nodes), 0)
    #initialize probed --> nodes that have been probed 
    probed = []
    
    
    # keep track of node IDs
    node_id = list(sample.nodes())
    
    
    # original graph
    sample_original = sample
    count = 0
    distinct = None

    while(count<budget):
        # with probability e0 we "explore" and with probability 1-e0 we exploit
        # outer level bandit
        if (random.randrange(0, 1)<e0):
            #explore
            if (random.randrange(0, 1)<0.5): # inner bandit
                #densification
                select, sample, node_id, reward_total, reward_obs, reward, distinct, C = densification(e1, sample, data, node_id,
                                                                                            reward_total, reward_obs, reward, distinct, C, probed)
                # update probe
                if (select not in probed):
                    probed.append(select)
                count = count+1 # one probe completed 
                
                # update reward_den 
                if distinct:
                    reward_den = (reward_den*num_den + 1)/(num_den + 1)
                     
                else: 
                    reward_den = (reward_den*num_den)/(num_den + 1)
                    
                num_den = num_den + 1

            else:
                #expansion --> randomly selected newly added unprobed node
                select, sample, node_id, reward_total, reward_obs, reward, distinct, C = expansion(probed, sample, data, node_id, reward_total, reward_obs, reward, distinct, C, probed)
                # update probe 
                if (select not in probed):
                    probed.append(select)
                count = count+1 # one probe completed 

                # update reward_exp 
                if distinct:
                    reward_exp = (reward_exp*num_exp + 1)/(num_exp + 1)

                else: 
                    reward_exp = (reward_exp*num_exp)/(num_exp + 1)

                num_exp = num_exp + 1
    
        else:
            #exploit --> pick expansion versus densification based on average rewards of each 
            if (max(reward_exp, reward_den) == reward_exp): #expansion
                # select newly added unprobed node
                select, sample, node_id, reward_total, reward_obs, reward, distinct, C = expansion(sample, data, node_id, reward_total, reward_obs, reward, distinct, C, probed)
                # update probe  
                if (select not in probed):
                    probed.append(select)
                count = count+1 # one probe completed 
                
                # update reward_exp 
                if distinct:
                    reward_exp = (reward_exp*num_exp + 1)/(num_exp + 1)
                    
                else: 
                    reward_exp = (reward_exp*num_exp)/(num_exp + 1)
                    
                num_exp = num_exp + 1
                        
            else: 
                #densification
                select, sample, node_id, reward_total, reward_obs, reward, distinct, C = densification(e1, sample, data, node_id,
                                                                                            reward_total, reward_obs, reward, distinct, C, probed)
                # update probe
                if (select not in probed):
                    probed.append(select)
                count = count+1 # one probe completed 
                
                # update reward_den 
                if distinct:
                    reward_den = (reward_den*num_den + 1)/(num_den + 1)
                     
                else: 
                    reward_den = (reward_den*num_den)/(num_den + 1)
                    
                num_den = num_den + 1
                        
    return sample
    

In [193]:
# Sampling: random edge 

def randEdge(data, size):
    sample = nx.Graph()
    edges = [e for e in data.edges]
    node=0
    while(node<size):
        sample_edge = random.choice(edges)
        if (sample_edge not in [e for e in sample.edges]):
            sample.add_edge(sample_edge[0], sample_edge[1])
            node = nx.number_of_nodes(sample)

    return sample

In [199]:
# test with random graph 

graph = nx.erdos_renyi_graph(1000, 0.05, seed=None, directed=False)
sample = randEdge(graph, 250)

s = eWGX(.2, .2, sample, graph, 500)


(0, 0)
(0, 1)
(0, 0)
(0, 1)
(0, 0)
(0, 1)
(0, 0)
(0, 1)
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
(0, 0)
(0, 1)
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
Solve for m failed, setting m = 1
s  = 1.0
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
(0, 0)
(0, 1)
(0, 0)
(0, 1)
(2, 0)
(2, 1)
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
Solv

Solve for m failed, setting m = 1
s  = 1.0
(2, 1)
(2, 2)
(1, 1)
(1, 2)
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
(1, 0)
(1, 1)
(0, 1)
(0, 2)
(0, 0)
(0, 1)
(0, 0)
(0, 1)
(0, 1)
(0, 2)
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
(1, 0)
(1, 1)
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
(0, 1)
(0, 2)
Correctly solved for m
s  = 0.5
(0, 0)
(0, 1)
Correctly solved for m
s  = 0.5
(0, 0)
(0, 1)
(1, 0)
(1, 1)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 1)
(0, 2)
(0, 1)
(0, 2)
(0, 0)
(0, 1)
(0, 0)
(0, 1)
(0, 1)
(0, 2)
(0, 1)
(0, 2)
Correctly solved for m
s  = 0.5
(1, 1)
(1, 2)
Correctly solved for m
s  = 0.3333333333333333
Correctly solved for m
s  = 0.3333333333333333
(1, 2)
(1, 3)
(0, 0)
(0, 1)
Solve for m failed, setting m = 1
s  = 1.0
Solve for m failed, setting m = 1
s  = 1.0
Solve for m failed, setting m = 1
s  = 1.0
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(1, 0)
(1, 1)
(1, 0)
(1, 1)
(0, 1)
(0, 2)
(0, 1)
(0, 2)
(1, 3)
(1, 4)
Solve for m failed, setting m = 1

In [ ]:
list(sample.neighbors(429))

In [ ]:
def my_function(x):
    return (1-exp(-m))/m

def makePoly(arr):
    def fn(x):
        return sum(c*x**p for p,c in enumerate(arr))
    return fn

my_func = makePoly([6, 2])
my_func(3)    # returns 12

def dx(fn, x, delta=0.001):
    return (fn(x+delta) - fn(x))/delta

def solve(fn, value, x=0.5, maxtries=1000, maxerr=0.00001):
    for tries in range(maxtries):
        err = fn(x) - value
        if abs(err) < maxerr:
            return x
        slope = dx(fn, x)
        x -= err/slope
    raise ValueError('no solution found')
    
solve(my_func, .2)  